In [1]:
# import required classes
import numpy as np
import os
from skimage.io import imread, imsave
from PIL import Image, ImageDraw, ImageFont
from keras.models import load_model
import threading

from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model 
from keras.applications.mobilenetv2 import MobileNetV2
from keras.applications.densenet import DenseNet121
from keras.applications.nasnet import NASNetMobile

import sys
if "E:\\git\\keras-resnet" not in sys.path:
    sys.path.append("E:\\git\\keras-resnet")
    print(sys.path)

import resnet
import keras.applications
    
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

# configure memory allocate for tensorflow backend
config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.6
config.gpu_options.allow_growth = True  #dynamically grow the memory used on the GPU
set_session(tf.Session(config=config))

Using TensorFlow backend.


['E:\\git\\models\\research', 'E:\\git\\models\\research\\slim', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\python36.zip', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\DLLs', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\lib', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36', '', 'C:\\Users\\hoang\\AppData\\Roaming\\Python\\Python36\\site-packages', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\lib\\site-packages', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\hoang\\.ipython', 'E:\\git\\keras-resnet']


In [2]:
def build_MobileNetV2_model():
    net = MobileNetV2(input_shape=(512, 512, 3), include_top=None, weights=None, classes=nb_classes)
    x = net.output

    outputs = []
    multi_output = 4

    for i in range(multi_output):
        pool = GlobalAveragePooling2D()(x)
        dense = Dense(nb_classes, activation='softmax', use_bias=True)(pool)
        outputs.append(dense)

    model = Model(net.inputs, outputs=outputs, name='mobilenetv2')
    return model

def build_DenseNet121_model():
    net = DenseNet121(input_shape=(512, 512, 3), include_top=False, weights=None, classes=nb_classes)
    x = net.output

    outputs = []
    multi_output = 4
    for i in range(multi_output):
        pool = GlobalAveragePooling2D()(x)
        dense = Dense(nb_classes, activation='softmax')(pool)
        outputs.append(dense)

    model = Model(net.inputs, outputs=outputs, name='densenet')
    return model

def build_NasnetMobile_model():
    net = NASNetMobile(input_shape=(512, 512, 3), include_top=False, weights=None, classes=nb_classes)
    x = net.output

    outputs = []
    multi_output = 4
    for i in range(multi_output):
        avg = GlobalAveragePooling2D()(x)
        dense = Dense(nb_classes, activation='softmax')(avg)
        outputs.append(dense)

    model = Model(net.inputs, outputs=outputs, name='nasnet')
    return model

In [3]:
nb_classes = 3

# Resnet18
# voa_model = load_model('C:\\Users\\hoang\\WorkingSpace\\TrainingModels\\keras\\results\\model_resnet18_view-offset-altitude_12.hdf5')
# cam_pitch_model = load_model('C:\\Users\\hoang\\WorkingSpace\\TrainingModels\\keras\\results\\model_resnet18_cam-pitch_transfer-learning_08.hdf5')
# model = resnet.ResnetBuilder.build_resnet_18_uav((3, 512, 512), nb_classes, multi_output=4)

#MobileNetV2
voa_model = load_model('C:\\Users\\hoang\\WorkingSpace\\TrainingModels\\keras\\results\\model_mobilenetv2_view-offset-altitude_08.hdf5')
cam_pitch_model = load_model('C:\\Users\\hoang\\WorkingSpace\\TrainingModels\\keras\\results\\model_mobilenetv2_cam-pitch_transfer-learning_09.hdf5')
model = build_MobileNetV2_model()

# DenseNet121
# voa_model = load_model('C:\\Users\\hoang\\WorkingSpace\\TrainingModels\\keras\\results\\model_densenet121_view-offset-altitude_11.hdf5')
# cam_pitch_model = load_model('C:\\Users\\hoang\\WorkingSpace\\TrainingModels\\keras\\results\\model_densenet121_cam_pitch_transfer-learning_09.hdf5')
# model = build_DenseNet121_model()

#NasnetMobile
# voa_model = load_model('C:\\Users\\hoang\\WorkingSpace\\TrainingModels\\keras\\results\\model_nasnetmobile_view-offset-altitude_08.hdf5')
# cam_pitch_model = load_model('C:\\Users\\hoang\\WorkingSpace\\TrainingModels\\keras\\results\\model_nasnetmobile_cam_pitch_transfer-learning_17.hdf5')
# model = build_NasnetMobile_model()

w1 = voa_model.get_weights()
w2 = cam_pitch_model.get_weights()
w = w1 + w2[-2:]

model.set_weights(w)    

In [4]:
def write_probs_on_image(vv_prob, height_prob, offset_prob, view_prob, vertical_angle, image_array, img_path):
    # initialise the drawing context with
    # the image object as background
    img = Image.fromarray(image_array)
    draw = ImageDraw.Draw(img)
    # create font object with the font file and specify
    # desired size
    font = ImageFont.truetype('Roboto-Medium.ttf', size=15)
    
    # OFFSET
    (x, y) = (235, 450)
    color = 'rgb(0, 255, 255)' 
    draw.text((x, y), 'Offset: ', fill=color, font=font)
    
    (x, y) = (290, 450)
    color = 'rgb(255, 255, 0)' 
    draw.text((x, y), 'C', fill=color, font=font)
    draw.text((x+15, y), str("{0:.3f}".format(offset_prob[0])), fill=color, font=font)

    (x, y) = (355, 450)
    color = 'rgb(0, 255, 0)' #  color
    draw.text((x, y), 'M', fill=color, font=font)
    draw.text((x+15, y), str("{0:.3f}".format(offset_prob[1])), fill=color, font=font)

    (x, y) = (420, 450)
    color = 'rgb(255, 0, 255)' #  color
    draw.text((x, y), 'F', fill=color, font=font)
    draw.text((x+15, y), str("{0:.3f}".format(offset_prob[2])), fill=color, font=font)
    
    # VIEW
    (x, y) = (235, 420)
    color = 'rgb(0, 255, 255)' 
    draw.text((x, y), 'View: ', fill=color, font=font)
    
    (x, y) = (290, 420)
    color = 'rgb(255, 255, 0)' 
    draw.text((x, y), 'L', fill=color, font=font)
    draw.text((x+15, y), str("{0:.3f}".format(view_prob[0])), fill=color, font=font)

    (x, y) = (355, 420)
    color = 'rgb(0, 255, 0)' #  color
    draw.text((x, y), 'S', fill=color, font=font)
    draw.text((x+15, y), str("{0:.3f}".format(view_prob[1])), fill=color, font=font)

    (x, y) = (420, 420)
    color = 'rgb(255, 0, 255)' #  color
    draw.text((x, y), 'R', fill=color, font=font)
    draw.text((x+15, y), str("{0:.3f}".format(view_prob[2])), fill=color, font=font)
    
    # HEIGHT
    # starting position of the message
    (x, y) = (300, 300)
    color = 'rgb(0, 255, 255)' 
    draw.text((x, y), 'Altitude: ', fill=color, font=font)
    
    (x, y) = (300, 320)
    color = 'rgb(255, 255, 0)' 
    draw.text((x, y), 'H', fill=color, font=font)
    draw.text((x+20, y), str("{0:.3f}".format(height_prob[2])), fill=color, font=font)

    (x, y) = (300, 340)
    color = 'rgb(0, 255, 0)' #  color
    draw.text((x, y), 'M', fill=color, font=font)
    draw.text((x+20, y), str("{0:.3f}".format(height_prob[1])), fill=color, font=font)

    (x, y) = (300, 360)
    color = 'rgb(255, 0, 255)' #  color
    draw.text((x, y), 'L', fill=color, font=font)
    draw.text((x+20, y), str("{0:.3f}".format(height_prob[0])), fill=color, font=font)
    
    # VERTICAL_VIEW
    # starting position of the message
    (x, y) = (400, 300)
    color = 'rgb(0, 255, 255)' 
    draw.text((x, y), 'Vertical-View: ', fill=color, font=font)
    
    (x, y) = (400, 320)
    color = 'rgb(255, 255, 0)' 
    draw.text((x, y), 'U', fill=color, font=font)
    draw.text((x+20, y), str("{0:.3f}".format(vv_prob[2])), fill=color, font=font)

    (x, y) = (400, 340)
    color = 'rgb(0, 255, 0)' #  color
    draw.text((x, y), 'M', fill=color, font=font)
    draw.text((x+20, y), str("{0:.3f}".format(vv_prob[1])), fill=color, font=font)

    (x, y) = (400, 360)
    color = 'rgb(255, 0, 255)' #  color
    draw.text((x, y), 'D', fill=color, font=font)
    draw.text((x+20, y), str("{0:.3f}".format(vv_prob[0])), fill=color, font=font)

    # VERTICAL_VIEW
    # starting position of the message
    (x, y) = (400, 250)
    color = 'rgb(0, 255, 255)' 
    draw.text((x, y), 'Vertical-Angle: ', fill=color, font=font)
    color = 'rgb(255, 160, 16)' 
    draw.text((x+20, y+20), str("{0:.1f}".format(vertical_angle)), fill=color, font=font)
    
    
    # save the edited image
    #print(img_path)
    img.save(img_path)

f = 'E:\\UAV_drone\\data\\2019-01-04-6groups_120fov_height_normal\\val\\medium\\img_0_0_1546559335497177700.png'
# f = 'img_0_0_1546722126943805600.png'
img = imread(f) 
img_rgb = img[:,:,:3]
img_norm = img_rgb / 255.0
img_final = np.reshape(img_norm,[1,512,512,3])
probs = model.predict(img_final)
print (probs)
offset_prob = probs[0][0]
view_prob = probs[1][0]
height_prob = probs[2][0]
pitch_prob = probs[3][0]
print(pitch_prob)
print(height_prob)
print(offset_prob)
print(view_prob)
img_path = os.path.join('.' , 'prob_'+'.png')
write_probs_on_image(pitch_prob, height_prob, offset_prob, view_prob, 0, img_rgb, img_path)


[array([[1.0000000e+00, 4.4525567e-08, 1.9623884e-18]], dtype=float32), array([[3.7051745e-07, 9.9999964e-01, 1.6780914e-09]], dtype=float32), array([[2.928667e-05, 9.999707e-01, 1.784223e-14]], dtype=float32), array([[4.6562172e-06, 9.9870706e-01, 1.2883403e-03]], dtype=float32)]
[4.6562172e-06 9.9870706e-01 1.2883403e-03]
[2.928667e-05 9.999707e-01 1.784223e-14]
[1.0000000e+00 4.4525567e-08 1.9623884e-18]
[3.7051745e-07 9.9999964e-01 1.6780914e-09]


In [5]:
import random

rain_values = [0, 0.5, 1]
snow_values = [0, 0.5, 1]
fog_values = [0, 0.25, 0.5, 0.75, 1]
mapleleaf_values = [0, 0, 0, 0, 0.25, 0.5]
dust_values = [0, 0.1, 0.25, 0.5, 0.75, 1]
def setWeather(rain=0, snow=0, fog=0, mapleleaf=0, dust=0, isRandom=False):
    client.simEnableWeather(True)   
    if isRandom:
        r = random.choice(rain_values)
        s = random.choice(snow_values)
        f = random.choice(fog_values)
        l = random.choice(mapleleaf_values)
        d = random.choice(dust_values)   
    else:
        r = rain
        s = snow
        f = fog
        l = mapleleaf
        d = dust
    
    print ("r{}, s{}, f{}, l{}, d{}".format(r,s,f,l,d))
    client.simSetWeatherParameter(airsim.WeatherParameter.Rain, r)
    client.simSetWeatherParameter(airsim.WeatherParameter.Snow, s)
    client.simSetWeatherParameter(airsim.WeatherParameter.Fog, f)
    client.simSetWeatherParameter(airsim.WeatherParameter.MapleLeaf, l)
    client.simSetWeatherParameter(airsim.WeatherParameter.Dust, d)

In [6]:
weatherList = [[1.0, 1.0, 0.25, 0.0, 0.25],
               [0.0, 0.0, 0.75, 0.0, 0.0],
               [0.0, 0.0, 0.25, 0.75, 0.25],
               [0.0, 0.0, 0.0, 0.0, 0.75],
               [0.5, 0.5, 0.5, 0.5, 0.5]]


FLYING_LOG_DIR = "E:\\UAV_drone\\Logs\\flying_recording_play_arounddfsdfdfsfsdfgfdfdd"

if not os.path.exists(FLYING_LOG_DIR):
    os.makedirs(FLYING_LOG_DIR)

### Detection preparation

In [7]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

PATH_TO_FROZEN_GRAPH = 'C:\\Users\\hoang\\WorkingSpace\\TrainingModels\\object_detection\\models\\frozen_inference_graph.pb'
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
    

### Sanity test the model

In [8]:
import common_functions as control
from common_types import *
import airsim
from airsim.types import *

import pprint
import time
import math

def transform_input_for_detection(responses, detection_graph):
    response = responses[0]
    img1d_rgba = np.frombuffer(response.image_data_uint8, dtype=np.uint8)
    img4d_rgba = img1d_rgba.reshape(512, 512, 4)
    img3d_rbg = img4d_rgba[:, :, :3] #512*512*3
    
    img3d_rbg_reshape = np.reshape(img3d_rbg,[1,512,512,3])
    ### insulator detection ###
    capture_points = detect_objects(img3d_rbg_reshape, detection_graph, 4)
    
    return capture_points

def transform_input(responses, vertical_angle, i, f, recording=False, vertical_view_only = False):
   
    response = responses[0]
    img1d_rgba = np.frombuffer(response.image_data_uint8, dtype=np.uint8)
    img4d_rgba = img1d_rgba.reshape(512, 512, 4)
    img3d_rbg = img4d_rgba[:, :, :3] #512*512*3
    
    img3d_rbg_reshape = np.reshape(img3d_rbg,[1,512,512,3])
    img_final = img3d_rbg_reshape/255.
    
    #height_prob = vertical_model.predict(img_final)[0]
    probs = model.predict(img_final)
    offset_prob = probs[0][0]
    view_prob = probs[1][0]
    height_prob = probs[2][0]  
    pitch_prob = probs[3][0]
    
    if (recording): 
        img_name = 'prob_'+str(i)+'.png'
        img_path = os.path.join(FLYING_LOG_DIR , img_name)
        #imsave(img, img3d_rbg)
        write_probs_on_image(pitch_prob, height_prob, offset_prob, view_prob, -vertical_angle, img3d_rbg, img_path)
    
    ### Mast detection ###
    mast_detections = detect_objects(img3d_rbg_reshape, detection_graph, 4) #replace 4(insulator) to 1(mast) when mast detection is ok
    
    
    return (pitch_prob, height_prob, offset_prob, view_prob, mast_detections)

def getPitchRollYawInRad():
    return airsim.to_eularian_angles(client.simGetGroundTruthKinematics().orientation)

def getZ():
    return client.simGetGroundTruthKinematics().position.z_val

def toDegree(rad):
    return rad*180.0/math.pi

def toRad(angle):
    return angle*math.pi/180.0

def moveByYawPitch(cur_yaw_rad, new_yaw_rad, z, cur_z, angle, velocity, headless= False, file=None, recording=False): 
    yaw = cur_yaw_rad + new_yaw_rad
    
    gv = (velocity - z*math.tan(angle)) * math.cos(angle)
    z_new = (velocity-z*math.tan(angle)) * math.sin(angle) + z/math.cos(angle)
       
    vx = gv*math.cos(yaw)
    vy = gv*math.sin(yaw)
      
    if recording == True:
        file.write('headless {}, vx {:.3f}, vy {:.3f}, z_new {:.3f}, yaw {}\n'.format(headless, vx, vy, z_new, -toDegree(new_yaw_rad)))
    
    if headless == True:
        client.moveByVelocityZAsync(vx, vy, cur_z+z_new, 0.2, drivetrain=DrivetrainType.ForwardOnly, yaw_mode = YawMode(False,-toDegree(new_yaw_rad))).join()
    else:
        client.moveByVelocityZAsync(vx, vy, cur_z+z_new, 0.2, drivetrain=DrivetrainType.ForwardOnly, yaw_mode = YawMode(False,0)).join()

def capture_images(client, object_type_name, i):
    responses = client.simGetImages([airsim.ImageRequest("0", airsim.ImageType.Scene, False, False)])
    response = responses[0]

    # get numpy array
    img1d = np.frombuffer(response.image_data_uint8, dtype=np.uint8) 

    # reshape array to 4 channel image array H X W X 4
    img_rgb = img1d.reshape(response.height, response.width, 4)

    # original image is fliped vertically
    img_rgb = np.flipud(img_rgb)

    # write to png 
    airsim.write_png(os.path.normpath('captures/' + object_type_name + str(i) +'.png'), img_rgb) 

def detect_objects(img3d_rbg_reshape, detection_graph, object_id):
    output_dict = run_inference_for_single_image(img3d_rbg_reshape, detection_graph)
    n = output_dict['num_detections']
        
    capture_points = []
    for j in range (0,n):
        if output_dict['detection_scores'][j] > 0.5: 
            print ("iter-{}: {}, {}, {}\n".format(i, output_dict['detection_classes'][j], 
                                         output_dict['detection_scores'][j], 
                                         output_dict['detection_boxes'][j]))
            class_id = output_dict['detection_classes'][j]
            if class_id == object_id:
                capture_points.append(output_dict['detection_boxes'][j])
                
    return capture_points

def cal_yaw_change_for_detection(coords):
    x = (coords[1] + coords[3])/2
    print (x)
    new_yaw_rad = math.atan((0.5-x)/(0.577*0.5)) # tan(30degree) = 0.577
    new_yaw_degree = toDegree (new_yaw_rad_1)
    print (new_yaw_degree) 
    return new_yaw_degree

['E:\\git\\models\\research', 'E:\\git\\models\\research\\slim', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\python36.zip', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\DLLs', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\lib', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36', '', 'C:\\Users\\hoang\\AppData\\Roaming\\Python\\Python36\\site-packages', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\lib\\site-packages', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\hoang\\.ipython', 'E:\\git\\keras-resnet', 'E:\\git\\AirSim\\PythonClient']
-- ['E:\\git\\models\\research', 'E:\\git\\models\\research\\slim', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\python36.zip', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\DLLs', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\lib', 'C:\\Users\\hoang\\AppData\\Local\\

In [9]:
# connect to the AirSim simulator
client = airsim.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)
# takeoff
client.takeoffAsync().join()
client.moveToZAsync(-1, 1).join()

vertical_angle = 0
client.simSetCameraOrientation("0", airsim.to_quaternion(toRad(-45 - vertical_angle), 0, 0)) # down 45 degree 
print('test')
# client.simSetCameraOrientation("0", airsim.to_quaternion(-0.5236, 0, 0)) # down 30 degree 
# client.simSetCameraOrientation("0", airsim.to_quaternion(-0.08727, 0, 0)) # down 15 degree 


Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

test


In [10]:
import math
def toDegree(rad):
    return rad*180.0/math.pi

print('test')
x = 0.35
new_yaw_rad_1 = math.atan((0.5-x)/(0.577*0.5)) # tan(30degree) = 0.577
new_yaw_degree_1 = toDegree (new_yaw_rad_1)
print (new_yaw_degree_1)

test
27.471304980740648


In [11]:
#client.simSetCameraOrientation("0", airsim.to_quaternion(-0.5235, 0, 0)) #radians
#home_location = client.simGetGroundTruthKinematics().position
name = os.path.join(FLYING_LOG_DIR, "logs.txt")
f = open(name, "a+")

recording = False


change_pitch_count = 0
total_z_angle_prob = 0

user_input = []

# spawn a new thread to wait for input 
def get_user_input(user_input):
    input("Press enter to stop flying: ")
    user_input.append(None)

mythread = threading.Thread(target=get_user_input, args=(user_input,))
mythread.daemon = True
mythread.start()

adjust_camera_enable = True

i = 0
while True:    
#     if (i%50==0):
#         (r, s, fo, l, d) = weatherList[int(i/50)%4]
#         setWeather(r, s, fo, l, d)
        
    _, _, cur_yaw_rad = getPitchRollYawInRad()
    cur_yaw_degree = toDegree(cur_yaw_rad)
    cur_z = getZ()
    f.write('{}, cur_yaw {:.3f}, cur_z {:.3f}\n'.format(i, cur_yaw_degree, cur_z))
    
    responses = client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.Scene, False, False)])
    pitch_prob, height_prob, offset_prob, view_prob, mast_detections  = transform_input(responses, vertical_angle, i, f, recording)      

    ### objects detected ###
    if len(mast_detections) > 0:     
        new_yaw_degree_tmp = cal_yaw_change_for_detection(mast_detections[0])
                      
        # stop, rotate to mast
        client.moveByVelocityZAsync(0, 0, getZ()-0.25, 0.005, drivetrain=DrivetrainType.ForwardOnly, yaw_mode = YawMode(False,0)).join()
        client.hoverAsync().join()
        client.rotateToYawAsync(cur_yaw_degree - new_yaw_degree_tmp, 1, 1).join()
        client.hoverAsync().join()
        
        # capture mast
        capture_images(client, "mast", i)
        
        # TODO find other objects (now is a insulator)
        responses = client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.Scene, False, False)])
        capture_points = transform_input_for_detection(responses, detection_graph)
        j=0
        for point in capture_points:
            # rotate to insulator    
            client.rotateToYawAsync(cur_yaw_degree - new_yaw_degree_tmp, 1, 1).join()
            client.hoverAsync().join()
            # capture insulator
            capture_images(client, "mast"+str(i)+"ins", j)
            j+=1
        
        # done, back to the inital angle
        client.rotateToYawAsync(cur_yaw_degree, 1, 1).join()
        
    ################################    
    
    z_angle_prob = pitch_prob[0] - pitch_prob[2] # down - up 

    # check and raise the camera
    if adjust_camera_enable == True:
        if (abs(z_angle_prob)>0.5):        
            change_pitch_count += 1
            total_z_angle_prob += z_angle_prob
            if (recording):
                f.write('{}, change_pitch_count {},  total_z_angle_prob {}, vertical_angle is {:.3f}\n'.format(i,change_pitch_count, total_z_angle_prob, vertical_angle))

            if (change_pitch_count >= 3):

                vertical_angle += -(total_z_angle_prob/change_pitch_count)*10
                client.simSetCameraOrientation("0", airsim.to_quaternion(toRad(-45 - vertical_angle), 0, 0)) 

                change_pitch_count = 0
                total_z_angle_prob = 0

        else:
            change_pitch_count = 0
            total_z_angle_prob = 0
            
    # navigate
    z_offset_2 = 0.4*(height_prob[0] - height_prob[2]) # low - high
    new_z = z_offset_2
    
    if ((height_prob[0] > 0.95 or height_prob[2] >0.95) and (adjust_camera_enable == False or abs(vertical_angle) >= 25.0)):
        #double move higher or lower
        new_z *= 1.5
    
    latoff_angle = 10 * (offset_prob[0] - offset_prob[2]) # left - right
    
    view_angle = 0
    view_angle = 10 * (view_prob[0] - view_prob[2])
    headless = False 
    
    if (view_prob[1] >= 0.99):
        headless = True       
    
    new_yaw_degree =  view_angle + latoff_angle 
    
    if (recording):
        f.write('{}, added_yaw {:.3f}, added_z {:.3f}, z_angle_prob {:.3f}\n'.format(i, new_yaw_degree, new_z, z_angle_prob))
        f.write('{}, view_angle {:.3f}, latoff_angle {:.3f}\n'.format(i, view_angle, latoff_angle))
          
    
    if (headless == False and (new_yaw_degree > 10.5 or new_yaw_degree < -10.5)):
        if (recording):
            f.write('****turning with a BIG ANGLE\n')
        #double rotate in this case
        client.rotateToYawAsync(cur_yaw_degree + new_yaw_degree, 1, 1).join()
   
    if adjust_camera_enable == True:
        velocity = 0.25*pitch_prob[1] + 0.25*offset_prob[1] + 0.25*view_prob[1] + 0.25*height_prob[1]
    else:
        velocity = 0.333*offset_prob[1] + 0.333*view_prob[1] + 0.333*height_prob[1]
    new_yaw_rad = toRad(new_yaw_degree)
    _, _, cur_yaw_rad = getPitchRollYawInRad()
    
    cur_yaw_degree = toDegree(cur_yaw_rad)
    cur_z = getZ()
    f.write('{}, cur_yaw {:.3f}, cur_z {:.3f}\n'.format(i, cur_yaw_degree, cur_z))
    
    moveByYawPitch(cur_yaw_rad, new_yaw_rad, -new_z, cur_z, toRad(vertical_angle), velocity, headless, file=f, recording=True)
    
    i+=1
    if user_input:
        break 

print ("stopped at ", i)
#work around to make drone stop
client.moveByVelocityZAsync(0, 0, getZ()-0.25, 0.005, drivetrain=DrivetrainType.ForwardOnly, yaw_mode = YawMode(False,0)).join()
client.hoverAsync().join()

f.close()

iter-0: 4, 0.9854394197463989, [0.3662891  0.30443874 0.42115596 0.3672025 ]

iter-0: 4, 0.8818758726119995, [0.36949944 0.20409991 0.42158544 0.2688345 ]

iter-0: 3, 0.7687592506408691, [0.00761697 0.00428891 0.36481962 0.98157525]

0.33582061529159546
27.471304980740648
iter-0: 3, 0.8581703305244446, [5.4784119e-04 2.6605219e-02 3.5587966e-01 1.0000000e+00]

iter-0: 4, 0.6923450827598572, [0.36290306 0.5006753  0.4125551  0.552125  ]

iter-1: 4, 0.9938367009162903, [0.3329078 0.3225479 0.387353  0.3789705]

iter-1: 4, 0.8398850560188293, [0.33104092 0.21851754 0.38447452 0.29026884]

iter-1: 3, 0.8146003484725952, [0.00521189 0.0060015  0.33476162 0.9637742 ]

0.35075920820236206
27.471304980740648
iter-1: 3, 0.9883679747581482, [0.42615458 0.26324654 0.590116   0.5448009 ]

iter-1: 4, 0.9743846654891968, [0.46813676 0.44555482 0.5265161  0.5007308 ]

iter-1: 4, 0.9328581094741821, [0.39203778 0.2687595  0.4385983  0.3314991 ]

iter-1: 4, 0.8902230858802795, [0.42348328 0.34527117 0.

In [12]:
# client.moveByVelocityAsync(-1, 0, 0, 5, drivetrain=DrivetrainType.ForwardOnly, yaw_mode = YawMode(False,0)).join()
# client.moveByVelocityAsync(-1, -1, 0, 5, drivetrain=DrivetrainType.ForwardOnly, yaw_mode = YawMode(False,0)).join()
# client.moveByVelocityAsync(-1, -1, -1, 5, drivetrain=DrivetrainType.ForwardOnly, yaw_mode = YawMode(False,0)).join()

# client.moveByAngleZAsync(pitch=1, roll=0, yaw=1)
# client.simGetObjectPose("H_mast_ins_brown_4_white_0")

### Intervention code

In [13]:
f = open(name, "a+")
f.write('intervention, moving drone back to proper position')

50

In [14]:
# shilf right
new_yaw_rad = toRad(90)
_, _, cur_yaw_rad = getPitchRollYawInRad()
cur_yaw_degree = toDegree(cur_yaw_rad)
cur_z = getZ()
name = os.path.join(FLYING_LOG_DIR, "logs.txt")

moveByYawPitch(cur_yaw_rad, new_yaw_rad, 0, cur_z, toRad(vertical_angle), velocity, True, file=f, recording=True)


In [15]:
client.moveByVelocityZAsync(0, 0, getZ()-0.25, 0.005, drivetrain=DrivetrainType.ForwardOnly, yaw_mode = YawMode(False,0)).join()
client.hoverAsync().join()

In [16]:
# rotate
_, _, cur_yaw_rad = getPitchRollYawInRad()
cur_yaw_degree = toDegree(cur_yaw_rad)
new_yaw_degree =  90
client.rotateToYawAsync(cur_yaw_degree + new_yaw_degree, 1, 1).join()

In [17]:
# move higher 1 m
client.moveToZAsync(getZ()-1, 1).join()

KeyboardInterrupt: 

In [ ]:
# adjust camera
vertical_angle = 0
client.simSetCameraOrientation("0", airsim.to_quaternion(toRad(-45 - vertical_angle), 0, 0)) # down 45 degree 

In [ ]:
f.write('done intervention'.format(i, cur_yaw_degree, cur_z))
f.close()

### End intervention